# Data Collection

## Part 1: Web Scraping

In the following section, I scrape the Mayo Clinic's Symptoms and Causes pages under all of their indexed diseases and conditions.

In [1]:
# import the necessary libraries
import sys
import requests
from bs4 import BeautifulSoup
from string import ascii_uppercase as upp
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

The first step is to get the long list of symptoms-causes URLs that are linked from the Mayo Clinic's indexed Diseases and Conditions lookup. I begin by saving the url up to and including the query for the letter, but not the letter itself. I also save the root of the URL for use later.

In [2]:
# get URLs: diseases-conditions pages organized in an alphabetical index, with one # entry
url = 'https://www.mayoclinic.org/diseases-conditions/index?letter='
root = 'https://www.mayoclinic.org'

Next, I write a function to extract the URLs of the pages I want from the letter index pages.

In [3]:
def extract(letter, addresses):
    index = url + letter # url of index page for this letter
    get = requests.get(index).content
    soup = BeautifulSoup(get, "lxml")
    # get list of articles on index page
    within = soup.find_all(class_ = "index content-within")
    for elm in within:
        # gets letter's articles as strings in a list
        links = re.findall("(?<=a\shref=\").*?(?=\">)", str(elm))
        # for each of the links
        for page in range(len(links)): 
            full = root + links[page] # symptoms-causes URL
            if addresses.count(full)<1:
                addresses.append(full)

I use the function to extract the URL from the # index page, the only index item that isn't listed under a capital letter of the alphabet. This helps me to ensure the function is behaving correctly without scraping too much.

In [4]:
addresses = []
extract("0", addresses)
addresses

['https://www.mayoclinic.org/diseases-conditions/digeorge-syndrome/symptoms-causes/syc-20353543']

Now that I know it works, I can extract the other URLs:

In [5]:
for letter in upp:
    extract(letter, addresses)

Now it's time to put these URLs into an initial dataframe that I will merge with the relevant Spider and Moz data.

In [6]:
mayo_data = pd.DataFrame(addresses, columns=["url"])
mayo_data.head()

,url
0,https://www.mayoclinic.org/diseases-conditions...
1,https://www.mayoclinic.org/diseases-conditions...
2,https://www.mayoclinic.org/diseases-conditions...
3,https://www.mayoclinic.org/diseases-conditions...
4,https://www.mayoclinic.org/diseases-conditions...


In [7]:
mayo_data.shape

(1183, 1)

## Part 2: SEO Spider Data

#### Raw Source Data: https://drive.google.com/open?id=1vlTTVOf3L2TnJxRma4TyJPCsgKpVvM19

Below are all of the columns provided by SEO Spider, not all of which will be useful for my purposes.

In [8]:
raw = pd.read_csv("symptoms-causes.csv")
raw.head(1)

,Address,Content,Status Code,Status,Indexability,Indexability Status,Title 1,Title 1 Length,Title 1 Pixel Width,Meta Description 1,...,Outlinks,Unique Outlinks,External Outlinks,Unique External Outlinks,Hash,Response Time,Last Modified,Redirect URL,Redirect Type,URL Encoded Address
0,https://www.mayoclinic.org/diseases-conditions...,text/html; charset=utf-8,200,OK,NaN,NaN,Congenital heart disease in adults - Symptoms ...,70,643,Learn about treatments and complications of he...,...,112,83,70,48,NaN,NaN,NaN,NaN,NaN,https://www.mayoclinic.org/diseases-conditions...


In [9]:
raw.columns

Index(['Address', 'Content', 'Status Code', 'Status', 'Indexability',
       'Indexability Status', 'Title 1', 'Title 1 Length',
       'Title 1 Pixel Width', 'Meta Description 1',
       'Meta Description 1 Length', 'Meta Description 1 Pixel Width',
       'Meta Keyword 1', 'Meta Keywords 1 Length', 'H1-1', 'H1-1 length',
       'H1-2', 'H1-2 length', 'H2-1', 'H2-1 length', 'H2-2', 'H2-2 length',
       'Meta Robots 1', 'X-Robots-Tag 1', 'Meta Refresh 1',
       'Canonical Link Element 1', 'rel="next" 1', 'rel="prev" 1',
       'HTTP rel="next" 1', 'HTTP rel="prev" 1', 'Size (bytes)', 'Word Count',
       'Text Ratio', 'Crawl Depth', 'Link Score', 'Inlinks', 'Unique Inlinks',
       '% of Total', 'Outlinks', 'Unique Outlinks', 'External Outlinks',
       'Unique External Outlinks', 'Hash', 'Response Time', 'Last Modified',
       'Redirect URL', 'Redirect Type', 'URL Encoded Address'],
      dtype='object')

I select which columns I want to keep:

In [36]:
trimmed = raw[["URL Encoded Address", 'H1-1', 'H1-1 length', "Meta Description 1", 'Meta Description 1 Length', 'Size (bytes)', "Word Count", "Inlinks", "Unique Inlinks", 'Outlinks', 'Unique Outlinks', 'External Outlinks',
       'Unique External Outlinks']]
trimmed.head()

,URL Encoded Address,H1-1,H1-1 length,Meta Description 1,Meta Description 1 Length,Size (bytes),Word Count,Inlinks,Unique Inlinks,Outlinks,Unique Outlinks,External Outlinks,Unique External Outlinks
0,https://www.mayoclinic.org/diseases-conditions...,Congenital heart disease in adults,34,Learn about treatments and complications of he...,114,60740,2005,53,29,112,83,70,48
1,https://www.mayoclinic.org/diseases-conditions...,Pulmonary fibrosis,18,"Pulmonary fibrosis — Learn about the symptoms,...",154,54587,2083,36,18,83,53,67,45
2,https://www.mayoclinic.org/diseases-conditions...,Epilepsy,8,"Learn about epilepsy symptoms, possible causes...",125,63278,2749,40,21,90,60,73,51
3,https://www.mayoclinic.org/diseases-conditions...,Cirrhosis,9,Cirrhosis is an advanced stage of scarring and...,147,55146,2008,35,17,82,53,67,45
4,https://www.mayoclinic.org/diseases-conditions...,Heart arrhythmia,16,Learn about common heart disorders that can ca...,103,68502,3397,54,34,94,65,67,45


...and convert them into more coding-friendly formats:

In [38]:
# lowercase, spaces to underscores
new_colnames = [x.lower() for x in trimmed.columns]
new_colnames = [x.replace(' ', '_') for x in new_colnames]

# replace in original dataframe
cleaned = trimmed
cleaned.columns = new_colnames

# replace individual column names that need modifying
cleaned = cleaned.rename(columns = {'url_encoded_address' : 'url'})
cleaned = cleaned.rename(columns = {'h1-1': 'header'})
cleaned = cleaned.rename(columns = {'h1-1_length': 'header_len'})
cleaned = cleaned.rename(columns = {'meta_description_1' : 'meta'})
cleaned = cleaned.rename(columns = {'meta_description_1_length' : 'meta_len'})
cleaned = cleaned.rename(columns = {'size_(bytes)' : 'bytes'})

# reformat headers
lower = [x.lower() for x in cleaned["header"]]
cleaned["header"] = lower

cleaned.head(3)

,url,header,header_len,meta,meta_len,bytes,word_count,inlinks,unique_inlinks,outlinks,unique_outlinks,external_outlinks,unique_external_outlinks
0,https://www.mayoclinic.org/diseases-conditions...,congenital heart disease in adults,34,Learn about treatments and complications of he...,114,60740,2005,53,29,112,83,70,48
1,https://www.mayoclinic.org/diseases-conditions...,pulmonary fibrosis,18,"Pulmonary fibrosis — Learn about the symptoms,...",154,54587,2083,36,18,83,53,67,45
2,https://www.mayoclinic.org/diseases-conditions...,epilepsy,8,"Learn about epilepsy symptoms, possible causes...",125,63278,2749,40,21,90,60,73,51


In [39]:
cleaned.shape

(1199, 13)

## Part 3: Moz Data

In [13]:
moz = pd.read_csv("moz-top-pages.csv")
moz.head()

,URL,Title,Total Links,PA,Linking Domains to Page,HTTP Status Code,Outbound Domains from Page,Outbound Links from Page
0,www.mayoclinic.org/,\r\n Mayo Clinic - Mayo Clinic,1008399,74,34849,200.0,10,31
1,www.mayoclinic.org/healthy-lifestyle/nutrition...,\r\n\tWater: How much should you drink every d...,11319,68,3523,200.0,10,37
2,www.mayoclinic.org/healthy-lifestyle/fitness/i...,\r\n\tExercise: 7 benefits of regular physical...,7961,67,2779,200.0,10,37
3,www.mayoclinic.org/healthy-lifestyle/nutrition...,\r\n\tMediterranean diet for heart health - Ma...,8152,67,2675,200.0,10,37
4,www.mayoclinic.org/healthy-lifestyle/stress-ma...,\r\n\tStress symptoms: Effects on your body an...,5819,67,2365,200.0,10,36


In [14]:
moz.columns

Index(['URL', 'Title', 'Total Links', 'PA', 'Linking Domains to Page',
       'HTTP Status Code', 'Outbound Domains from Page',
       'Outbound Links from Page'],
      dtype='object')

In [15]:
moz = moz.drop(columns = ["Title", "HTTP Status Code"])

In [16]:
# lowercase, spaces to underscores
colnames = [x.lower() for x in moz.columns]
colnames = [x.replace(' ', '_') for x in colnames]

In [17]:
# replace in original dataframe
moz.columns = colnames

In [18]:
# clarify page authority column
moz = moz.rename(columns = {'pa': 'page_auth'})

In [19]:
moz.head()

,url,total_links,page_auth,linking_domains_to_page,outbound_domains_from_page,outbound_links_from_page
0,www.mayoclinic.org/,1008399,74,34849,10,31
1,www.mayoclinic.org/healthy-lifestyle/nutrition...,11319,68,3523,10,37
2,www.mayoclinic.org/healthy-lifestyle/fitness/i...,7961,67,2779,10,37
3,www.mayoclinic.org/healthy-lifestyle/nutrition...,8152,67,2675,10,37
4,www.mayoclinic.org/healthy-lifestyle/stress-ma...,5819,67,2365,10,36


In [20]:
# make sure url format matches
urls = []
for row in range(len(moz["url"])):	
    urls.append("https://"+moz["url"][row])

In [21]:
moz["url"]=urls

In [22]:
moz.head()

,url,total_links,page_auth,linking_domains_to_page,outbound_domains_from_page,outbound_links_from_page
0,https://www.mayoclinic.org/,1008399,74,34849,10,31
1,https://www.mayoclinic.org/healthy-lifestyle/n...,11319,68,3523,10,37
2,https://www.mayoclinic.org/healthy-lifestyle/f...,7961,67,2779,10,37
3,https://www.mayoclinic.org/healthy-lifestyle/n...,8152,67,2675,10,37
4,https://www.mayoclinic.org/healthy-lifestyle/s...,5819,67,2365,10,36


## Part 4: Merge 3 Datasets

In merging the datasets, I am paring down the data quite a lot. I perform two inner merges, first with the scraped data and Spider data, then again with the top-ranking Mayo Clinic pages. Since Moz provides the 500 best-ranking pages for a site domain, not all of which are Symptoms and Causes pages, our rows reduce to a mere 216.

In [40]:
data = pd.merge(mayo_data, cleaned, on='url', how='inner')
data = pd.merge(data, moz, on='url', how='inner')
data.head(3)

,url,header,header_len,meta,meta_len,bytes,word_count,inlinks,unique_inlinks,outlinks,unique_outlinks,external_outlinks,unique_external_outlinks,total_links,page_auth,linking_domains_to_page,outbound_domains_from_page,outbound_links_from_page
0,https://www.mayoclinic.org/diseases-conditions...,atrial fibrillation,19,"Find out about atrial fibrillation, a heart co...",152,68770,2732,31,19,100,71,75,52,5801,62,882,10,34
1,https://www.mayoclinic.org/diseases-conditions...,achilles tendinitis,19,Achilles tendinitis — Comprehensive overview c...,103,45417,1438,9,4,70,43,69,46,1242,60,397,10,32
2,https://www.mayoclinic.org/diseases-conditions...,gastroesophageal reflux disease (gerd),38,"GERD, in which stomach acid moves into the eso...",140,52830,1555,16,8,85,52,71,47,6148,62,1091,10,35


In [41]:
data.shape

(222, 18)

The scraped data includes some duplicates, since some conditions are listed in multiple languages or featured multiple times under different names on different index pages.

In [42]:
# get rid of any duplicate URLs
data = data[data["url"].duplicated() == False]

In [43]:
data.shape

(216, 18)

In [44]:
#check for any NaNs in the dataset
data.isnull().values.any()

False

## Part 5: More Web Scraping
I want the date of publication from the pages I'll be working with, and now that I have a much more limited selection of pages to scrape, this won't overwhelm the Mayo Clinic domain.


In [46]:
dates = []

for page in data["url"]:
    content = requests.get(page).content # page content
    file = BeautifulSoup(content, "lxml") # in lxml
    date = file.find("div", class_='pubdate') # read date
    if(date!=None):
        match = re.findall("(?<=\\r\\n).*?(?=\\r\\n)", str(date.get_text()))[0].strip() # get rid of \r\n and spaces
        dates.append(match) # add to column list
    else:
        dates.append(None)

# add new column to dataframe
data['pub_date'] = dates
data.head()

,url,header,header_len,meta,meta_len,bytes,word_count,inlinks,unique_inlinks,outlinks,unique_outlinks,external_outlinks,unique_external_outlinks,total_links,page_auth,linking_domains_to_page,outbound_domains_from_page,outbound_links_from_page,pub_date
0,https://www.mayoclinic.org/diseases-conditions...,atrial fibrillation,19,"Find out about atrial fibrillation, a heart co...",152,68770,2732,31,19,100,71,75,52,5801,62,882,10,34,"June 20, 2019"
1,https://www.mayoclinic.org/diseases-conditions...,achilles tendinitis,19,Achilles tendinitis — Comprehensive overview c...,103,45417,1438,9,4,70,43,69,46,1242,60,397,10,32,"Sept. 17, 2019"
2,https://www.mayoclinic.org/diseases-conditions...,gastroesophageal reflux disease (gerd),38,"GERD, in which stomach acid moves into the eso...",140,52830,1555,16,8,85,52,71,47,6148,62,1091,10,35,"April 03, 2020"
3,https://www.mayoclinic.org/diseases-conditions...,acne,4,"Learn more about this common skin condition, i...",144,59112,2067,11,6,85,57,71,48,4497,63,1290,10,34,"Feb. 18, 2020"
4,https://www.mayoclinic.org/diseases-conditions...,hiv/aids,8,Learn more about this potentially life-threate...,142,61851,2856,14,5,76,47,68,46,2793,60,575,10,33,"Feb. 13, 2020"


In [47]:
data.isnull().values.any()

False

In [49]:
# fix publication dates so they can be compared
import datetime
# jan, feb, aug, sept, oct, nov, dec
dat = []
for date in data["pub_date"]:
    if type(date)==str:
        if "." in date:
            if "Sept" in date: #special case: datetime recognizes "Sep" not "Sept"
                date = re.sub('t', '', date)
            datetime_ob = datetime.datetime.strptime(date, '%b. %d, %Y')
        else:
            datetime_ob = datetime.datetime.strptime(date, '%B %d, %Y')
        dat.append(datetime_ob)
    else:
        dat.append(None)

data.pub_date = dat

In [50]:
data.head()

,url,header,header_len,meta,meta_len,bytes,word_count,inlinks,unique_inlinks,outlinks,unique_outlinks,external_outlinks,unique_external_outlinks,total_links,page_auth,linking_domains_to_page,outbound_domains_from_page,outbound_links_from_page,pub_date
0,https://www.mayoclinic.org/diseases-conditions...,atrial fibrillation,19,"Find out about atrial fibrillation, a heart co...",152,68770,2732,31,19,100,71,75,52,5801,62,882,10,34,2019-06-20
1,https://www.mayoclinic.org/diseases-conditions...,achilles tendinitis,19,Achilles tendinitis — Comprehensive overview c...,103,45417,1438,9,4,70,43,69,46,1242,60,397,10,32,2019-09-17
2,https://www.mayoclinic.org/diseases-conditions...,gastroesophageal reflux disease (gerd),38,"GERD, in which stomach acid moves into the eso...",140,52830,1555,16,8,85,52,71,47,6148,62,1091,10,35,2020-04-03
3,https://www.mayoclinic.org/diseases-conditions...,acne,4,"Learn more about this common skin condition, i...",144,59112,2067,11,6,85,57,71,48,4497,63,1290,10,34,2020-02-18
4,https://www.mayoclinic.org/diseases-conditions...,hiv/aids,8,Learn more about this potentially life-threate...,142,61851,2856,14,5,76,47,68,46,2793,60,575,10,33,2020-02-13


In [51]:
# check the column value types
for col in data.columns:
    print(col, type(data[col][0]))

url <class 'str'>
header <class 'str'>
header_len <class 'numpy.int64'>
meta <class 'str'>
meta_len <class 'numpy.int64'>
bytes <class 'numpy.int64'>
word_count <class 'numpy.int64'>
inlinks <class 'numpy.int64'>
unique_inlinks <class 'numpy.int64'>
outlinks <class 'numpy.int64'>
unique_outlinks <class 'numpy.int64'>
external_outlinks <class 'numpy.int64'>
unique_external_outlinks <class 'numpy.int64'>
total_links <class 'numpy.int64'>
page_auth <class 'numpy.int64'>
linking_domains_to_page <class 'numpy.int64'>
outbound_domains_from_page <class 'numpy.int64'>
outbound_links_from_page <class 'numpy.int64'>
pub_date <class 'pandas._libs.tslibs.timestamps.Timestamp'>


## Part 6: Save dataset

In [53]:
data.to_csv('data.csv')